In [22]:
# Importing all libraries that will be used
from collections import deque
import heapq
from typing import Tuple, Callable, List, Any, Set, Deque, Dict

In [13]:
type State = Tuple[Tuple[int, ...], ...]
INF = 10**9

def bfs(s: State, t: State, sucessors: Callable[[State], List[State]]) -> List[State]:
    q: Deque[State] = deque()
    vis: Set[State] = set()
    par: Dict[State, State] = dict()

    # Handle start
    q.append(s)
    vis.add(s)
    par[s] = None

    while q:
        # Let's go level by level
        n = len(q)
        found = False
        while n:
            curr = q.popleft()

            # check if its the target state
            if curr == t:
                found = True
                break

            for next in sucessors(curr):
                if next not in vis:
                    vis.add(next)
                    q.append(next)
                    par[next] = curr

            n -= 1

        if found:
            break

    path: List[State] = []

    node = t

    while node:
        path.append(node)
        node = par[node]

    # reverse in place
    # path.reverse()

    # Replace by copy
    path = path[::-1]

    return path

In [34]:
def h_manhattan(state: State) -> int:
    ans = 0
    
    n, m = len(state), len(state[0])
    
    for i in range(n):
        for j in range(m):
            if state[i][j] == 0:
                continue
            
            val = state[i][j] - 1
            
            r, c = val // 3, val % 3
            
            ans += abs(i - r) + abs(j - c)
            
    return ans 

In [32]:
def reconstruct_path(parent, goal):
    path = []
    while goal:
        path.append(goal)
        goal = parent[goal]
    return path[::-1]


def greedy_best_first(
    start: State,
    goal: State,
    successors: Callable[[State], List[State]]
) -> List[State]:

    pq = []
    heapq.heappush(pq, (h_manhattan(start), start))

    parent = {start: None}
    visited: Set[State] = set()

    while pq:
        _, curr = heapq.heappop(pq)

        if curr in visited:
            continue

        visited.add(curr)

        if curr == goal:
            return reconstruct_path(parent, curr)

        for nxt in successors(curr):
            if nxt not in visited:
                parent[nxt] = curr
                heapq.heappush(pq, (h_manhattan(nxt), nxt))

    return []  # no path

In [3]:
def print_path(path: List[State]) -> None:
    n = len(path)
    for idx in range(n):
        if idx == n - 1:
            print(path[idx])
        else:
            print(path[idx], end=" --> ")

In [4]:
# Man, Goat, Wolf, and Cabbage Problem

# Initial state
s = (0, 0, 0, 0)
t = (1, 1, 1, 1)

In [5]:
def is_safe(state: Tuple) -> bool:
    (M, W, C, G) = state

    if M == 0:
        return ((G + W) != 2) and ((G + C) != 2)
    else:
        return ((G + W) != 0) and ((G + C) != 0)


# def sucessors(state: Tuple) -> List[Tuple]:
    # (G, C), and (G, W) can't be together

    # 2 cases, either man will go alone or will take someone with himself such that everything is safe
    (M, W, C, G) = state

    # Move Man to other side
    M = 1 - M

    next_states: List[Tuple] = []
    possible_states = [
        (M, W, C, G),
        (M, 1 - W, C, G),
        (M, W, 1- C, G),
        (M, W, C, 1 - G)
    ]
    
    for st in possible_states:
        if is_safe(st):
            next_states.append(st)
    
    return next_states

In [6]:
# path = bfs(s, t, sucessors)

# print_path(path)

In [30]:
S: State = (
    (1, 2, 3),
    (0, 4, 5),
    (6, 7, 8)
)

T: State = (
    (1, 2, 3),
    (4, 5, 6),
    (7, 8, 0)
)

# 5 / 3 -> rows = 1
# 5 // 3 -> cols = 2

# 6 / 3 -> row = 2
# 6 // 3 -> col = 0

In [16]:
def successors(state: State) -> List[State]:
    n, m = len(state), len(state[0])
    x, y = -1, -1

    # find blank (0)
    for i in range(n):
        for j in range(m):
            if state[i][j] == 9:
                x, y = i, j
                break

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    next_states: List[State] = []

    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < n and 0 <= ny < m:
            # make mutable copy
            board = [list(row) for row in state]

            # swap
            board[x][y], board[nx][ny] = board[nx][ny], board[x][y]

            # convert back to hashable state (immutable)
            next_states.append(tuple(tuple(row) for row in board))

    return next_states


In [33]:
path = greedy_best_first(S, T, successors)
print(len(path))

print_path(path)

0
